<img src="../images/GA-logo.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Singapore Housing Data and Kaggle Challenge

**Primary Learning Objectives:**

1. Creating and iteratively refining a regression model
2. Using Kaggle to practice the modeling process
3. Providing business insights through reporting and presentation.

We will clean the data and build a regression model based on Singapore Housing Dataset to predict the price of a house at sale.

This jupyter notebook focuses on testing our the models prepared in the 02_Feature_Engineering notebook, evaluate the performance and decide which model to put into production

----

### Contents:
- [Data and Preprocessor Import](#Data-and-Preprocessor-Import)
- [Model Tuning and Evaluation](#Model-Tuning-and-Evaluation)

In [1]:
import dill
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, PolynomialFeatures
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

from subsegment_model import SubsegmentModel

from model_metrics import test_metric # import user-defined function for model evaluation

----

## Data and Preprocessor Import

In [2]:
# import clean data
X = pd.read_csv('../datasets/X.csv', index_col='Unnamed: 0')
y = pd.read_csv('../datasets/y.csv', index_col='Unnamed: 0').squeeze() # convert y into a Series from a Dataframe to prevent errors

In [3]:
# define folder path for models
folder_path = '../models/'

In [4]:
# load preprocessors
preprocessor_A = dill.load(open(folder_path + 'preprocessor_A.sav', 'rb'))
preprocessor_B = dill.load(open(folder_path + 'preprocessor_B.sav', 'rb'))
preprocessor_C = dill.load(open(folder_path + 'preprocessor_C.sav', 'rb'))
preprocessor_D = dill.load(open(folder_path + 'preprocessor_D.sav', 'rb'))

In [5]:
# load regression transformer
lr_log_model = dill.load(open(folder_path + 'lr_log.sav', 'rb'))

----

## Model Tuning and Evaluation

In [6]:
# conduct train-test-split for model tuning and evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, stratify=X['planning_area_grouped'])

### Baseline Model

We establish the baseline model as a dummy regressor that always predicts the resale price as the median resale price of the training dataset (considering that the resale prices are skewed normal distributed)

In [7]:
# baseline model
base_model = DummyRegressor(strategy='median')

base_model.fit(X_train, y_train)

DummyRegressor(strategy='median')

In [8]:
# look at model performance in terms of r2_score
test_metric(base_model, X_train, X_test, y_train, y_test,
            metric='r2_score', cv=5)

Train R2_SCORE:           	-0.0391
5-Fold CV R2_SCORE:     	-0.0401
Test R2_SCORE:            	-0.0373


In [9]:
# look at model performance in terms of rmse
test_metric(base_model, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

Train RMSE:           	146187.1039
5-Fold CV RMSE:     	146254.7380
Test RMSE:            	145629.5692


Observation:

- R2 score is -0.04 and RMSE is $146,000
- This is not unexpected, we now see if our models perform better

### Model A (simple model without amenities)

In [10]:
# create pipeline to combine preprocessor with regressor
model_A = Pipeline(
    steps=[
        ('preproc', preprocessor_A),
        ('lr', LinearRegression())
    ]
)

In [11]:
# fit model
model_A.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  ['planning_area',
                                                   'full_flat_type']),
                                                 ('dummy',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function <lambda> at 0x0000020366652DC0>),
                                                  ['hdb_age', 'mid',
                                                   'floor_area_sqft',
                                                   'tranc_year',
                                                   'tranc_month'])],
                                   verbose_feature_names_out=False)),
                ('lr', LinearRegression())])

In [12]:
# look at model performance in terms of r2_score
test_metric(model_A, X_train, X_test, y_train, y_test,
            metric='r2_score', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train R2_SCORE:           	0.8513
5-Fold CV R2_SCORE:     	0.8510
Test R2_SCORE:            	0.8547


In [13]:
# look at model performance in terms of rmse
test_metric(model_A, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	55311.6321
5-Fold CV RMSE:     	55349.5897
Test RMSE:            	54510.5944


Observation:

- R2 score is 0.85 and consistent across train, cross validation and test datasets, so unlikely to have any overfitting or underfitting
- RMSE has improved by more than 200% to \$54.3k
- **Model A performs better than Baseline Model**


### Model A2 (model without amenities)

In [14]:
# create pipeline to combine preprocessor with regressor that logs resale price
model_A2 = Pipeline(
    steps=[
        ('preproc', preprocessor_A),
        ('regr', lr_log_model)
    ]
)

In [15]:
# fit model
model_A2.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  ['planning_area',
                                                   'full_flat_type']),
                                                 ('dummy',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function <lambda> at 0x0000020366652DC0>),
                                                  ['hdb_age', 'mid',
                                                   'floor_area_sqft',
                                                   'tranc_year',
                                                   'tranc_month'])],
                                   verbose_feature_names_out=False)),
                ('regr',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc 'exp'>,
                                            regressor=LinearRegression()))])

In [16]:
# look at model performance in terms of r2_score
test_metric(model_A2, X_train, X_test, y_train, y_test,
            metric='r2_score', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train R2_SCORE:           	0.8641
5-Fold CV R2_SCORE:     	0.8637
Test R2_SCORE:            	0.8680


In [17]:
# look at model performance in terms of rmse
test_metric(model_A2, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	52875.6925
5-Fold CV RMSE:     	52941.8808
Test RMSE:            	51946.1804


Observation:

- R2 score is 0.86 and consistent across train, cross validation and test datasets, so unlikely to have any overfitting or underfitting
- RMSE has improved further by \$30k
- **Model A2 performs better than Baseline Model and Model A**


### Model B (model with amenities)

In [18]:
# create pipeline to combine preprocessor with regressor
model_B = Pipeline(
    steps=[
        ('preproc', preprocessor_B),
        ('regr', lr_log_model)
    ]
)

In [20]:
# fit model
model_B.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  ['planning_area',
                                                   'full_flat_type']),
                                                 ('imp', KNNImputer(),
                                                  ['mall_nearest_distance']),
                                                 ('dummy',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function <lambda> at 0x0000020366652E50>),
                                                  ['hdb_age', 'mid',
                                                   'floor_area_sqft',
                                                   'tranc_year', 'tranc_month',
                                                   'hawker_nearest_distance',
                                                   'hawker_within_2km',
                                                   'mrt_nearest_distance',
                                                   'bus_interchange',
                                                   'mrt_interchange'])],
                                   verbose_feature_names_out=False)),
                ('regr',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc 'exp'>,
                                            regressor=LinearRegression()))])

In [21]:
# look at model performance in terms of r2_score
test_metric(model_B, X_train, X_test, y_train, y_test,
            metric='r2_score', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train R2_SCORE:           	0.8953
5-Fold CV R2_SCORE:     	0.8949
Test R2_SCORE:            	0.8982


In [22]:
# look at model performance in terms of rmse
test_metric(model_B, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	46412.5343
5-Fold CV RMSE:     	46501.4039
Test RMSE:            	45623.5491


Observation:

- R2 score further improved to 0.90, a 4% improvement and consistent across train, cross validation and test datasets
- RMSE has improved further as well by \\$10k to $46.2k
- **Model B performs better than Baseline Model and Model A and A2**


### Model C (model with amenities and interaction)

In [23]:
# create pipeline to combine preprocessor with regressor
model_C = Pipeline(
    steps=[
        ('preproc', preprocessor_C),
        ('regr', lr_log_model)
    ]
)

In [24]:
# fit model
model_C.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  ['planning_area',
                                                   'full_flat_type']),
                                                 ('imp', KNNImputer(),
                                                  ['mall_nearest_distance']),
                                                 ('poly_hawk',
                                                  PolynomialFeatures(include_bias=False,
                                                                     interaction_only=True),
                                                  ['hawker_nearest_distance',
                                                   'hawker_within_2km']),
                                                 ('poly_mrt',
                                                  P...
                                                   'mrt_interchange',
                                                   'bus_interchange']),
                                                 ('dummy',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function <lambda> at 0x0000020366652F70>),
                                                  ['hdb_age', 'mid',
                                                   'floor_area_sqft',
                                                   'tranc_year',
                                                   'tranc_month'])],
                                   verbose_feature_names_out=False)),
                ('regr',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc 'exp'>,
                                            regressor=LinearRegression()))])

In [25]:
# look at model performance in terms of r2_score
test_metric(model_C, X_train, X_test, y_train, y_test,
            metric='r2_score', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train R2_SCORE:           	0.8954
5-Fold CV R2_SCORE:     	0.8950
Test R2_SCORE:            	0.8984


In [26]:
# look at model performance in terms of rmse
test_metric(model_C, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	46384.0558
5-Fold CV RMSE:     	46475.9258
Test RMSE:            	45569.5645


Observation:

- R2 score marginally improved by 0.01%, consistent across train, cross validation and test datasets
- RMSE has improved further as well by \\$100
- **Model C performs very marginally better than Model B**
- It is therefore arguable that Model B should be chosen because of lower complexity and thus lower risk of overfitting
- To better assess and address the risk of overfitting, we will include regularisation in Model C and examine the results


### Model CR (model with amenities, interaction and regularisation)

We will add in both L1 and L2 regularisation to remove variables that does not contribute significantly to predictive power and control overfitting

In [27]:
# create regressor with regularisation thru ElasticNet
enet_log_model = TransformedTargetRegressor(
    regressor=ElasticNet(),
    func=np.log,
    inverse_func=np.exp
)

In [28]:
# create pipeline to join preprocessor and regressor
model_CR = Pipeline(
    steps=[
        ('preproc', preprocessor_C),
        ('regr', enet_log_model)
    ]
)

In [29]:
# check the names of the parameters to set
model_CR['regr'].get_params()

{'check_inverse': True,
 'func': <ufunc 'log'>,
 'inverse_func': <ufunc 'exp'>,
 'regressor__alpha': 1.0,
 'regressor__copy_X': True,
 'regressor__fit_intercept': True,
 'regressor__l1_ratio': 0.5,
 'regressor__max_iter': 1000,
 'regressor__normalize': 'deprecated',
 'regressor__positive': False,
 'regressor__precompute': False,
 'regressor__random_state': None,
 'regressor__selection': 'cyclic',
 'regressor__tol': 0.0001,
 'regressor__warm_start': False,
 'regressor': ElasticNet(),
 'transformer': None}

#### 1st Round of Grid Search

We break the gridsearch down into multiple rounds to reduce computing load, as each model fit can take up to 1min

The first round of Grid Search will be broad with 10 steps of l1_ratio and 10 steps of alpha. Max iterations is kept to the default 1,000, which will result in some model fits not converging but that is okay for the first round since the exact coefficients are not that important

**WARNING: this round of Grid Search takes significant amounts of time. Suggest to skip if time is limited.**

In [51]:
# define hyperparameters for regularisation to search through
model_CR_params = {
    'regr__regressor__l1_ratio': np.linspace(0, 1, 10),
    'regr__regressor__alpha': np.logspace(0.0001, 100, 10),
    'regr__regressor__max_iter': 1000
}

In [42]:
model_CR_grid = GridSearchCV(
    estimator=model_CR,
    param_grid=model_CR_params,
    scoring='neg_mean_squared_error'
)

**WARNING**: Run the next batch of cells only if you wish to rerun the results of the 1st round of Grid Search. This can take up to 20min 

In [43]:
%%time
model_CR_grid.fit(X_train, y_train)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.290e+02, tolerance: 8.228e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the

GridSearchCV(estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['planning_area',
                                                                          'full_flat_type']),
                                                                        ('imp',
                                                                         KNNImputer(),
                                                                         ['mall_nearest_distance']),
                                                                        ('poly_hawk',
                                                                         PolynomialFeatures(include_bias=False,
                                                                                            interaction_only=True),
                                                                         ['hawker_nearest_distance',
                                                                          'hawker_with...
             param_grid={'regr__regressor__alpha': array([1.00023029e+000, 1.29181404e+011, 1.66839930e+022, 2.15476543e+033,
       2.78291537e+044, 3.59418146e+055, 4.64194510e+066, 5.99514926e+077,
       7.74283492e+088, 1.00000000e+100]),
                         'regr__regressor__l1_ratio': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])},
             scoring='neg_mean_squared_error')

In [46]:
# examine best estimator hyperparameters
model_CR_grid.best_estimator_

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  ['planning_area',
                                                   'full_flat_type']),
                                                 ('imp', KNNImputer(),
                                                  ['mall_nearest_distance']),
                                                 ('poly_hawk',
                                                  PolynomialFeatures(include_bias=False,
                                                                     interaction_only=True),
                                                  ['hawker_nearest_distance',
                                                   'hawker_within_2km']),
                                                 ('poly_mrt',
                                                  P...
                                                   'bus_interchange']),
                                                 ('dummy',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function <lambda> at 0x00000160BC7F14C0>),
                                                  ['hdb_age', 'mid',
                                                   'floor_area_sqft',
                                                   'tranc_year',
                                                   'tranc_month'])],
                                   verbose_feature_names_out=False)),
                ('regr',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc 'exp'>,
                                            regressor=ElasticNet(alpha=1.0002302850208247,
                                                                 l1_ratio=0.0)))])

In [52]:
# exmaine cv results
pd.DataFrame(model_CR_grid.cv_results_).head(20)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regr__regressor__alpha,param_regr__regressor__l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,15.938490,0.320265,0.983503,0.087144,1.00023,0.0,"{'regr__regressor__alpha': 1.0002302850208247,...",-4.615772e+09,-4.840872e+09,-4.448160e+09,-4.638772e+09,-4.691334e+09,-4.646982e+09,1.265956e+08,1
1,3.963501,0.114323,0.960527,0.082660,1.00023,0.111111,"{'regr__regressor__alpha': 1.0002302850208247,...",-5.803035e+09,-5.966600e+09,-5.574871e+09,-5.802274e+09,-5.810979e+09,-5.791552e+09,1.250734e+08,2
2,4.051527,0.083418,0.973262,0.094357,1.00023,0.222222,"{'regr__regressor__alpha': 1.0002302850208247,...",-6.990674e+09,-7.116362e+09,-6.701975e+09,-6.965087e+09,-6.970407e+09,-6.948901e+09,1.352667e+08,3
3,4.056419,0.159688,0.962851,0.075037,1.00023,0.333333,"{'regr__regressor__alpha': 1.0002302850208247,...",-7.719347e+09,-7.828392e+09,-7.380154e+09,-7.686784e+09,-7.692402e+09,-7.661416e+09,1.496323e+08,4
4,4.117488,0.139240,1.007544,0.074278,1.00023,0.444444,"{'regr__regressor__alpha': 1.0002302850208247,...",-8.467043e+09,-8.545499e+09,-8.093002e+09,-8.407386e+09,-8.406152e+09,-8.383816e+09,1.540750e+08,5
5,4.024889,0.078934,0.958908,0.089324,1.00023,0.555556,"{'regr__regressor__alpha': 1.0002302850208247,...",-8.807899e+09,-8.858267e+09,-8.407659e+09,-8.722028e+09,-8.708893e+09,-8.700949e+09,1.566612e+08,6
6,4.059613,0.139560,0.996004,0.038778,1.00023,0.666667,"{'regr__regressor__alpha': 1.0002302850208247,...",-9.190514e+09,-9.211221e+09,-8.762103e+09,-9.077070e+09,-9.051776e+09,-9.058537e+09,1.606193e+08,7
7,4.086559,0.140345,0.978579,0.075847,1.00023,0.777778,"{'regr__regressor__alpha': 1.0002302850208247,...",-9.613986e+09,-9.603456e+09,-9.155575e+09,-9.471778e+09,-9.434013e+09,-9.455761e+09,1.659458e+08,8
8,4.109124,0.111653,0.986705,0.068502,1.00023,0.888889,"{'regr__regressor__alpha': 1.0002302850208247,...",-1.004606e+10,-1.003380e+10,-9.585154e+09,-9.905070e+09,-9.854584e+09,-9.884932e+09,1.669221e+08,9
9,3.990823,0.144947,0.975496,0.093173,1.00023,1.0,"{'regr__regressor__alpha': 1.0002302850208247,...",-1.004895e+10,-1.004686e+10,-9.586272e+09,-9.922478e+09,-9.886368e+09,-9.898187e+09,1.690300e+08,10


In [53]:
test_metric(model_CR_grid.best_estimator_, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.290e+02, tolerance: 8.228e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the

Train RMSE:           	68157.5312
5-Fold CV RMSE:     	68162.4439
Test RMSE:            	68590.2361


Observation:

- Best alpha is around 1, with top 10 models all having the same alpha of 1. We will search between 0.1 and 5 in the second round
- Best l1_ratio is 0, i.e. Ridge regression given that all the cv results show a fall in performance as l1_ratio increases to 1.0 regardless of the alpha value. We will adjust the model to Ridge regression and tune for the best alpha for faster computation
- RMSE scores are worse than original Model C without regularisation, possibly because of reaching max iterations reaching before model is fully optimised. We will raise max iterations in next round of Grid Search

#### 2nd Round of Grid Search

We break the gridsearch down into multiple rounds to reduce computing load, as each model fit can take up to 1min

The second round of Grid Search zooms in to the best_estimator_ from the first round and searches the hyperparameters around this best_estimator_

Max iterations is also raised to 15,000 (default for Ridge regressor) to get a better estimate of the coefficients

In [30]:
# create regressor with regularisation thru ElasticNet
rr_log_model = TransformedTargetRegressor(
    regressor=Ridge(),
    func=np.log,
    inverse_func=np.exp
)

In [31]:
# create pipeline to join preprocessor and regressor
model_CR = Pipeline(
    steps=[
        ('preproc', preprocessor_C),
        ('regr', rr_log_model)
    ]
)

In [32]:
# define hyperparameters for regularisation to search through
model_CR_params = {
    'regr__regressor__alpha': np.linspace(0.1, 5.0, 20),
}

In [33]:
model_CR_grid = GridSearchCV(
    estimator=model_CR,
    param_grid=model_CR_params,
    scoring='neg_mean_squared_error',
    n_jobs=-2
)

In [34]:
%%time
model_CR_grid.fit(X_train, y_train)

CPU times: total: 3.97 s
Wall time: 1min 42s


GridSearchCV(estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['planning_area',
                                                                          'full_flat_type']),
                                                                        ('imp',
                                                                         KNNImputer(),
                                                                         ['mall_nearest_distance']),
                                                                        ('poly_hawk',
                                                                         PolynomialFeatures(include_bias=False,
                                                                                            interaction_only=True),
                                                                         ['hawker_nearest_distance',
                                                                          'hawker_with...
                                        TransformedTargetRegressor(func=<ufunc 'log'>,
                                                                   inverse_func=<ufunc 'exp'>,
                                                                   regressor=Ridge()))]),
             n_jobs=-2,
             param_grid={'regr__regressor__alpha': array([0.1       , 0.35789474, 0.61578947, 0.87368421, 1.13157895,
       1.38947368, 1.64736842, 1.90526316, 2.16315789, 2.42105263,
       2.67894737, 2.93684211, 3.19473684, 3.45263158, 3.71052632,
       3.96842105, 4.22631579, 4.48421053, 4.74210526, 5.        ])},
             scoring='neg_mean_squared_error')

In [35]:
model_CR_grid.best_estimator_

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  ['planning_area',
                                                   'full_flat_type']),
                                                 ('imp', KNNImputer(),
                                                  ['mall_nearest_distance']),
                                                 ('poly_hawk',
                                                  PolynomialFeatures(include_bias=False,
                                                                     interaction_only=True),
                                                  ['hawker_nearest_distance',
                                                   'hawker_within_2km']),
                                                 ('poly_mrt',
                                                  P...
                                                   'mrt_interchange',
                                                   'bus_interchange']),
                                                 ('dummy',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function <lambda> at 0x0000020366652F70>),
                                                  ['hdb_age', 'mid',
                                                   'floor_area_sqft',
                                                   'tranc_year',
                                                   'tranc_month'])],
                                   verbose_feature_names_out=False)),
                ('regr',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc 'exp'>,
                                            regressor=Ridge(alpha=0.1)))])

In [36]:
pd.DataFrame(model_CR_grid.cv_results_).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regr__regressor__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,22.506537,0.167044,0.855901,0.045706,0.1,{'regr__regressor__alpha': 0.1},-2.107488e+09,-2.113894e+09,-2.158370e+09,-2.295808e+09,-2.135258e+09,-2.162164e+09,6.916435e+07,1
1,11.838377,8.991073,0.815269,0.051900,0.357895,{'regr__regressor__alpha': 0.35789473684210527},-2.108592e+09,-2.114247e+09,-2.163256e+09,-2.306021e+09,-2.136134e+09,-2.165650e+09,7.277208e+07,2
2,5.225163,0.935515,0.793781,0.043594,0.615789,{'regr__regressor__alpha': 0.6157894736842106},-2.109844e+09,-2.114723e+09,-2.166949e+09,-2.312958e+09,-2.137123e+09,-2.168320e+09,7.508314e+07,3
3,6.442671,0.268307,0.822948,0.037057,0.873684,{'regr__regressor__alpha': 0.8736842105263158},-2.111067e+09,-2.115223e+09,-2.169847e+09,-2.317932e+09,-2.138071e+09,-2.170428e+09,7.664986e+07,4
4,4.529527,0.660274,0.729612,0.043177,1.131579,{'regr__regressor__alpha': 1.1315789473684212},-2.112219e+09,-2.115717e+09,-2.172194e+09,-2.321639e+09,-2.138951e+09,-2.172144e+09,7.775022e+07,5
5,5.037211,0.902660,0.706130,0.049170,1.389474,{'regr__regressor__alpha': 1.3894736842105266},-2.113298e+09,-2.116197e+09,-2.174144e+09,-2.324484e+09,-2.139767e+09,-2.173578e+09,7.853975e+07,6
6,5.946615,0.226455,0.733547,0.040244,1.647368,{'regr__regressor__alpha': 1.6473684210526318},-2.114309e+09,-2.116662e+09,-2.175798e+09,-2.326717e+09,-2.140525e+09,-2.174802e+09,7.911234e+07,7
7,8.356798,0.200574,0.724265,0.030363,1.905263,{'regr__regressor__alpha': 1.905263157894737},-2.115260e+09,-2.117114e+09,-2.177227e+09,-2.328501e+09,-2.141238e+09,-2.175868e+09,7.952796e+07,8
8,6.972616,0.883704,0.734241,0.066093,2.163158,{'regr__regressor__alpha': 2.1631578947368424},-2.116158e+09,-2.117554e+09,-2.178479e+09,-2.329947e+09,-2.141912e+09,-2.176810e+09,7.982668e+07,9
9,5.582123,1.267143,0.770907,0.032173,2.421053,{'regr__regressor__alpha': 2.421052631578948},-2.117011e+09,-2.117984e+09,-2.179591e+09,-2.331131e+09,-2.142555e+09,-2.177654e+09,8.003626e+07,10


In [37]:
test_metric(model_CR_grid.best_estimator_, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	46398.1849
5-Fold CV RMSE:     	46493.2377
Test RMSE:            	45580.7958


Observation:

- We note that the smaller alpha is, the better the RMSE, with the top ranked model having the smallest alpha of 0.1
- In addition, the RMSE score for this model is still slightly higher than Model C
- As such, **Model C with no regularisation will be preferred**, as it has lower RMSE than Model CR

### Model D (Subsegment models with amenities and interaction)

We will test fitting the last model where the dataset is split into segments by `planning_area` and each segment is fitted to a separate model

In [38]:
# define sub-segment model
model_D_segment = Pipeline(
                    steps=[
                        ('preproc', preprocessor_D),
                        ('regr', lr_log_model)
                    ]
                )

In [39]:
# create model with sub-segments
model_D = SubsegmentModel(model_D_segment, 'planning_area_grouped')

In [40]:
# fit model
model_D.fit(X_train, y_train)

In [42]:
# look at model performance in terms of r2_score
test_metric(model_D, X_train, X_test, y_train, y_test,
            metric='r2_score', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train R2_SCORE:           	0.9227
5-Fold CV R2_SCORE:     	0.9192
Test R2_SCORE:            	0.9224


C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packag

In [43]:
# look at model performance in terms of RMSE
test_metric(model_D, X_train, X_test, y_train, y_test,
            metric='rmse', cv=5)

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packag

Train RMSE:           	39978.7782
5-Fold CV RMSE:     	nan
Test RMSE:            	39996.2622


C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\fishe\Documents\0 Coding Things\GA Data Sci

Observation:

- **Model D outperforms Model C** by more than \\$6k, and lowers RMSE to \\$40k
- Cross validation for test failed because of unknown columns, likely due to small segmented dataset during training, but that is ok, since accuracy overall is still ok!
- Will deploy Model D for production